# To Do:
- figure out how to save graph locally

### references:


In [4]:
#printing: sys.stdout.write(f'\r  ...progress: {curRow * tilesY} / {tilesX * tilesY} ({pct:.2f}%)')

### Imports:

In [63]:
from lxml import etree
from lxml.etree import XMLSyntaxError
import pandas as pd
import sys
import os
import networkx as nx
from natsort import index_natsorted, order_by_index
from itertools import combinations

In [9]:
source = "dblp.xml" #dataset of publications
dtd = etree.DTD(file="dblp.dtd") #read DTD
publication_data = [] #fill this list with pairs of author/title
counter = 0
bad_titles = ["Home Page", "Title Page", "Welcome message.", "Workshop preface.", "Vorwort.", "The", "The ", "Session Summary.", "Reviewers.", "Program Committee.", 
    "Preface.", "Organizing Committee.", "Workshop Organization.", "Workshop Organizers' Message.", "Title Page.", "Steering Committee.", "Session details: Keynote Address.",
    "S"]
#iterate through nodes
for event, element in etree.iterparse(source, load_dtd=True):
    title = ""
    author_list = []
    year = 0
    #iterate through children: author, title, year, etc.
    for child in element:
        if child.tag == "author":
            counter+=1
        elif child.tag == "title":
            title = str(child.text)
            if title in bad_titles:
                title = "";
            # break
    if counter % 10000 == 0:
        sys.stdout.write(f'\r  progress:  {counter}  rows')
    if counter >= 1000000:
        print("\nbroke")
        break
#create dataframe
publication_df = pd.DataFrame(publication_data)

element.clear()

  progress:  1000000  rows
broke


## Working approach:

In [11]:
source = "dblp.xml" #dataset of publications
dtd = etree.DTD(file="dblp.dtd") #read DTD
publication_data = [] #fill this list with pairs of author/title
edge_list = [] #fill this with edges
counter = 0
bad_titles = ["Home Page", "Title Page", "Welcome message.", "Workshop preface.", "Vorwort.", "The", "The ", "Session Summary.", "Reviewers.", "Program Committee.", 
    "Preface.", "Organizing Committee.", "Workshop Organization.", "Workshop Organizers' Message.", "Title Page.", "Steering Committee.", "Session details: Keynote Address.",
    "S"]
#iterate through nodes
for event, element in etree.iterparse(source, load_dtd=True):
    title = ""
    author_list = []
    year = 0
    #iterate through children: author, title, year, etc.
    for child in element:
        if child.tag == "author":
            author_list.append(str(child.text))
        elif child.tag == "title":
            title = str(child.text)
            if title in bad_titles:
                title = ""; 
        elif child.tag == "year":
            year = int(child.text)
            for author in author_list:
                publication_data.append({"Author" : author.strip(), "Title" : title.strip(), "Year" : year}) #add author/title/year pair to data list
            if len(author_list) > 1:
                edge_list.append(author_list)
            counter += 1
            break
    if counter % 10000 == 0:
        sys.stdout.write(f'\r  progress:  {counter}  rows')
    if counter > 1000000:
        print("\nbroke")
        break
#create dataframe
publication_df = pd.DataFrame(publication_data)
edges_df = pd.DataFrame(edge_list)
element.clear()

  progress:  1000000  rowsbroke


In [ ]:
publication_df.sample(10)

### Converting Dataframe to CSV

In [13]:
publication_df.to_csv('data.csv')
edges_df.to_csv('edges.csv')
print(len(publication_df))
print(len(edges_df))

2951926
777398


### Converting CSV to graph: adding authors as nodes

In [15]:
publication_df = pd.read_csv("data.csv")
publication_df = publication_df.reindex(index=order_by_index(publication_df.index, index_natsorted(publication_df['Author'], reverse=False))) # sorting by author name
publication_df.to_csv('data.csv')


In [16]:
author_graph = nx.Graph()
publication_df = pd.read_csv("data.csv")
author_graph.add_node(publication_df["Author"][0], Publications = {publication_df["Title"][0]})
for row in range(1, len(publication_df)):
    author = publication_df["Author"][row]
    if author == publication_df["Author"][row-1]:
        author_graph.nodes[author]["Publications"].add(publication_df["Title"][row]) #adding title to list of publications from that author
    else:
        author_graph.add_node(author, Publications = {publication_df["Title"][row]}) #adding author to graph with title to attibute
print("Number of unique authors: " + str(len(author_graph.nodes)))

Number of unique authors: 965885


### Sorting CSV by title and url (not used)

In [ ]:
publication_df1 = pd.read_csv("data.csv")
# publication_df1 = publication_df1.reindex(index=order_by_index(publication_df1.index, index_natsorted(publication_df1['Url'], reverse=False)))
publication_df1 = publication_df1.reindex(index=order_by_index(publication_df1.index, index_natsorted(publication_df1['Title'], reverse=True)))
publication_df1.to_csv('data.csv')

C:\Users\mail4\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Adding edges for authors that share title (not used)

In [ ]:
row = 0
while (publication_df["Title"][row] != "None"): #for all elements with a title
# while row < len(publication_df2)-5:
    next = 1
    current_author = publication_df["Author"][row]
    co_authors = [current_author] #list of coauthors
    while publication_df["Title"][row] ==  publication_df["Title"][row + next]: #find all authors with same title
        if (current_author !=  publication_df["Author"][row + next]) and (publication_df2["Year"][row] ==  publication_df["Year"][row + next]): #make sure no edges between same author
            co_authors.append(publication_df["Author"][row + next]) #add to list of co-authors
            print(publication_df["Title"][row])
        next += 1
    author_graph.add_edges_from(list(combinations(co_authors, 2))) #add edges between all coauthors
    row += 1
print("Last row with a title: " + str(row))
print("Number of edges: " + str(len(author_graph.edges)))

### Adding edges for authors that share URL (not used)

In [19]:
row = 352356
while row < len(publication_df) - 1: #for all elements with no title
    next = 1
    co_authors = [publication_df["Author"][row]] #list of coauthors
    while publication_df["Url"][row] ==  publication_df["Url"][row + next]: #find all authors with same title
        if (publication_df["Author"][row] !=  publication_df["Author"][row + next]) and (publication_df2["Year"][row] ==  publication_df["Year"][row + next]): #make sure no edges between same author
            co_authors.append(publication_df["Author"][row + next]) #add to list of co-authors
        next += 1
    author_graph.add_edges_from(list(combinations(co_authors, 2))) #add edges between all coauthors
    row += 1
print("Number of edges: " + str(len(author_graph.edges)))

Number of edges: 566


### Adding edges based on CSV of edge list

In [ ]:
edges_list = pd.read_csv("edges.csv")

In [65]:
G = nx.Graph()
G.add_nodes_from([1, 2, 3])
print(G.nodes)
G.add_edges_from(list(combinations([1, 2, 3], 2)))
print(G.edges)

[1, 2, 3]
[(1, 2), (1, 3), (2, 3)]


In [72]:
count = 0
for row in edges_list.iterrows():
    i = 1
    coauthors = []
    while (type(row[1][i])) == str and (i < 15):
        coauthors.append(row[1][i])
        i += 1
    author_graph.add_edges_from(list(combinations(coauthors, 2)))
    count+=1
    if count % 1000 == 0:
        sys.stdout.write(f'\r  rows processed: {count}')

  rows processed: 777000

### Observations

In [73]:
print(str(sorted(author_graph.degree, key=lambda x: x[1], reverse=True)[0:10]) + "\n") # 10 authors with most coauthors

# google pagerank algorithm (error with this)
pageranks = nx.pagerank(author_graph) # A dictionary
print(sorted(pageranks, key=lambda x: x[1], reverse=True)[0:10]) # 10 authors with highest pageranks score (sharing publications with more important authors)

[('Wei Wang', 771), ('Yang Liu', 727), ('Lei Zhang', 726), ('Yu Zhang', 669), ('Wei Zhang', 653), ('Wei Li', 650), ('Jing Li', 634), ('Lei Wang', 621), ('Xin Li', 617), ('Wei Liu', 613)]

['Bülent Abali', 'Bülent Bolat', 'Bülent Çatay', 'Bülent Dal', 'Bülent Durak', 'Bülent Haznedar', 'Bülent Karasözen', 'Bülent Keskinler', 'Bülent Möller', 'Bülent Örencik']


### Save graph as a graphml

In [84]:
author_graph.nodes["Wei Liu"]
nx.write_gml(author_graph, "author_graph.gml")

{'Publications': {'3D SLAM for Scenes with Repetitive Texture Inside Tokamak Chamber.',
  '3D model retrieval based on orthogonal projections.',
  '50 nA, 1 V nanowatt resistor-free compact CMOS current references.',
  'A Conflict Detection Method for IPv6 Time-Based Firewall Policy.',
  'A Cross-Culture Study on Multimodal Emotion Recognition Using Deep Learning.',
  'A Data-Centric Storage Approach for Efficient Query of Large-Scale Smart Grid.',
  'A Density-Based Seed-Centric Community Detection Algorithm.',
  'A Diagnostic Knowledge Model of Wind Turbine Fault.',
  'A Honeycomb Artifacts Removal and Super Resolution Method for Fiber-Optic Images.',
  'A Key-Frame Selection Method for Semi-automatic 2D-to-3D Conversion.',
  'A Keyword-Driven Data Service Composition Sequence Generation Approach on Ad-Hoc Data Query.',
  'A Keyword-Driven Data Service Mashup Plan Generation Approach for Ad-Hoc Data Query.',
  'A Method for Evaluating the Sensitivity of Signal Features in Pattern Rec

### Read graphml back into networkX graph

In [21]:
author_graph = nx.read_graphml("author_graph.graphml")

In [2]:
df = pd.read_csv("data.csv")
count = 0
for i in range(len(df)):
    if df["Author"][i] == "Thomas Seidl 0001":
        count += 1
print(count)

C:\Users\mail4\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


99


In [ ]:
df = pd.read_csv("data.csv")
author_graph = nx.read_graphml("author_graph.graphml")
authors = df["Author"]
author_list = authors.sample(500)
subgraph = author_graph.subgraph(author_list)
nx.draw(subgraph, with_labels = True)